#기본코드



In [ ]:
!pip install -U finance-datareader
!pip install mpl_finance
import FinanceDataReader as fdr
df_krx = fdr.StockListing('KRX')


  Created wheel for mpl-finance: filename=mpl_finance-0.10.0-cp36-none-any.whl size=5509 sha256=b46bc85edd028ccf6436052ee1279c2277d45e33bfe0bf412262cf4989b2614f
  Stored in directory: /root/.cache/pip/wheels/67/db/24/04ddd06ec9a5e4500b44c13d8e95898f5b90b512303e385de8
Successfully built mpl-finance


In [ ]:
import mpl_finance 
from mpl_finance import candlestick2_ohlc
%matplotlib inline
import matplotlib.ticker as ticker
import numpy as np
import pandas_datareader.data as web
import datetime
from sklearn import linear_model
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')
import pandas as pd 
import FinanceDataReader as fdr

In [ ]:

name=input('주식 코드 기업명 ?')
#find 종목 코드 및 정보
df_krx[df_krx['Name'].str.contains(name)]


주식 코드 기업명 ?호텔신라


,Symbol,Name,Sector,Industry
99,008770,호텔신라,기타 상품 전문 소매업,"면세판매,관광숙박,외식사업,예식업"


In [ ]:
kospi = fdr.DataReader('KS11', '2019-01-01','2019-12-06')
삼성바이오=fdr.DataReader('207940', '2019-01-01','2019-12-06')
호텔신라=fdr.DataReader('008770', '2019-01-01','2019-12-06')
SK하이닉스=fdr.DataReader('000660', '2019-01-01','2019-12-06')
젬백스=fdr.DataReader('082270', '2019-01-01','2019-12-06')

젬백스

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2019-01-02,12250,12250,11600,11600,136937,-0.033333
2019-01-03,11600,11800,11400,11450,123640,-0.012931
2019-01-04,11200,11450,10950,11450,176723,0.000000
2019-01-07,11650,11650,11300,11450,121251,0.000000
2019-01-08,11550,11550,11000,11100,159859,-0.030568
...,...,...,...,...,...,...
2019-12-02,25250,26200,23500,25850,652789,0.017717
2019-12-03,25800,26850,23650,23900,1168109,-0.075435
2019-12-04,24500,26000,23900,25000,1192742,0.046025


In [ ]:
df =  pd.concat([삼성바이오['Close'], 호텔신라['Close'],SK하이닉스['Close'],
                젬백스['Close']],axis=1)
df.set_axis(['SSBL','HTSL','SKH','JBS'],axis=1,inplace=True)
#df.columns = (['SSBL','HTSL','SKH','JBS'],axis=1)
df.head(3)

,SSBL,HTSL,SKH,JBS
Date,,,,
2019-01-02,374000,73100,60600,11600
2019-01-03,376500,72700,57700,11450
2019-01-04,381000,71200,58300,11450


In [ ]:
#rev는 수익률
rev = df.diff(axis = 0 , periods = 1) / df.shift(periods = 1)
rev.dropna(inplace = True)
rev.head(3)
#diff (오늘종가 - 어제종가 )/ shift (어제종가) 
#내가 기간을 5일전을 한다 --- periods = 5 로 하면뎀 (shift도 5로 )

,SSBL,HTSL,SKH,JBS
Date,,,,
2019-01-03,0.006684,-0.005472,-0.047855,-0.012931
2019-01-04,0.011952,-0.020633,0.010399,0.000000
2019-01-07,0.018373,-0.002809,0.006861,0.000000


#1207 일별, 주별, 월별, 분기별 투자 수익률을 구하시오, 학번 20182890

##일별

In [ ]:
rev.shape[0]

230

In [ ]:
import numpy as np
rev= np.array(rev)
one =np.array(np.repeat(1,rev.shape[0]))
one.shape

(230,)

In [ ]:
exp = one.T@rev/rev.shape[0]
exp

array([0.0005059 , 0.00074571, 0.00148687, 0.00650823])

###수익률 공분산 구하기

In [ ]:
exp_repeat = np.tile(exp,(rev.shape[0],1))
exp_repeat.shape
A = rev.T - exp_repeat.T 
B = rev - exp_repeat
(A @ B)
cov_fn = (A @ B)/rev.shape[0]
cov_fn
#리스크
#1*1,2*2,3*3,4*4

array([[5.39425685e-04, 4.54526395e-05, 2.54202740e-05, 1.81396454e-04],
       [4.54526395e-05, 5.67423237e-04, 1.01976747e-04, 1.08643163e-04],
       [2.54202740e-05, 1.01976747e-04, 4.94425317e-04, 2.21164211e-04],
       [1.81396454e-04, 1.08643163e-04, 2.21164211e-04, 2.13313988e-03]])

###투자포트폴리오  기대수익률 , 위험분산 구하기

In [ ]:
import numpy as np 
pf = np.array([[ 0.25, 0.25, 0.25, 0.25] ,
               [0.1, 0.2, 0.2, 0.5]  ,
              [0.3 , 0.1, 0, 0.6 ]])


In [ ]:
pf@exp
#기대수익률


array([0.00231168, 0.00375122, 0.00413128])

In [ ]:
pf[0]@cov_fn@pf[0].T #p1의 분산
pf[1]@cov_fn@pf[1].T #p2의 분산
pf[2]@cov_fn@pf[2].T #p3의 분산


pf@cov_fn@pf.T
#위험분산

array([[0.00031891, 0.00043366, 0.00047662],
       [0.00043366, 0.00067625, 0.00075732],
       [0.00047662, 0.00075732, 0.00090322]])

기대수익률을 보고 선택한다면 3번포트폴리오를 선택한다. - 위험투자자들 추천

위험분산이 가장 낮은 것은 1번 포트폴리오를 선택한다. - 수동적인 투자자들 추천



###최소분산 포트폴리오



In [ ]:
#최소분산 포트폴리오
import numpy.linalg as la
a=la.inv(cov_fn)#역행렬 함수 inv
ab=np.array([np.repeat(1,rev.shape[1]),])
W=(ab@a)/(ab@a@ab.T)
W

array([[0.34966568, 0.28276109, 0.34742944, 0.02014379]])

In [ ]:
#W@exp.T 최소분산 포트폴리오의 기대 수익률
W@exp.T

array([0.00103543])

##주별

In [ ]:
sha = df.diff(axis = 0 , periods = 1) / df.shift(periods = 1)
sha.dropna(inplace = True)
import datetime 
sha['day'] = sha.index.weekday
sha['quarter'] = sha.index.quarter
sha['week'] = sha.index.week
sha['month'] = sha.index.month
sha['year'] = sha.index.year
sha.head(3)

,SSBL,HTSL,SKH,JBS,day,quarter,week,month,year
Date,,,,,,,,,
2019-01-03,0.006684,-0.005472,-0.047855,-0.012931,3,1,1,1,2019
2019-01-04,0.011952,-0.020633,0.010399,0.000000,4,1,1,1,2019
2019-01-07,0.018373,-0.002809,0.006861,0.000000,0,1,2,1,2019


In [ ]:
sha['week'] = sha['year'].astype(str)+'_'+ sha['week'].astype(str)
sha_w = sha.groupby(['week']).mean().iloc[:,0:4]
sha_w.head(3)
#주간

,SSBL,HTSL,SKH,JBS
week,,,,
2019_1,0.009318,-0.013052,-0.018728,-0.006466
2019_10,-0.001294,0.006901,-0.009558,-0.003569
2019_11,-0.017895,0.006339,0.004219,0.004446


In [ ]:
sha['month'] = sha['year'].astype(str)+'_'+ sha['month'].astype(str)
sha_m = sha.groupby(['month']).mean().iloc[:,0:4]
sha_m.head(3)
#월간

,SSBL,HTSL,SKH,JBS
month,,,,
2019_1,0.003222,0.002603,0.010002,0.003797
2019_10,0.012683,-0.004854,0.000068,0.040001
2019_11,-0.000488,0.004361,-0.000536,0.000002


In [ ]:
import numpy as np
sha_w= np.array(sha_w)
one1 =np.array(np.repeat(1,sha_w.shape[0]))
one1.shape

(49,)

In [ ]:
exp1 = one1.T@sha_w/sha_w.shape[0]
exp1

array([0.00056693, 0.00026588, 0.00088314, 0.00609417])

In [ ]:
exp_repeat1 = np.tile(exp1,(sha_w.shape[0],1))
exp_repeat1.shape
A1 = sha_w.T - exp_repeat1.T 
B1 = sha_w - exp_repeat1
(A1 @ B1)
cov_fn1 = (A1 @ B1)/sha_w.shape[0]
cov_fn1
#리스크
#1*1,2*2,3*3,4*4
#공분산

array([[1.09131351e-04, 7.51756891e-06, 2.53812163e-05, 3.41772671e-05],
       [7.51756891e-06, 1.06170402e-04, 3.88451027e-05, 4.13262311e-06],
       [2.53812163e-05, 3.88451027e-05, 1.15732753e-04, 4.35244533e-06],
       [3.41772671e-05, 4.13262311e-06, 4.35244533e-06, 5.79503139e-04]])

In [ ]:
pf@exp1
#주간 기대수익률

array([0.00195253, 0.00333358, 0.00385317])

In [ ]:
pf[0]@cov_fn1@pf[0].T #p1의 분산
pf[1]@cov_fn1@pf[1].T #p2의 분산
pf[2]@cov_fn1@pf[2].T #p3의 분산


pf@cov_fn1@pf.T
#주간 위험분산

array([[7.12093807e-05, 9.92247301e-05, 1.10457019e-04],
       [9.92247301e-05, 1.64381524e-04, 1.90476360e-04],
       [1.10457019e-04, 1.90476360e-04, 2.32755441e-04]])

In [ ]:
#최소분산 포트폴리오
import numpy.linalg as la
a1=la.inv(cov_fn1)#역행렬 함수 inv
ab1=np.array([np.repeat(1,sha_w.shape[1]),])
W1=(ab1@a1)/(ab1@a1@ab1.T)
W1

array([[0.35712247, 0.355053  , 0.22775224, 0.06007229]])

In [ ]:
#W@exp.T 최소분산 포트폴리오의 기대 수익률
W1@exp1.T

array([0.00086409])

#로그 기대수익률 가져오기

In [ ]:
re = df.diff(axis = 0 , periods = 1) / df.shift(periods = 1)

re.dropna(inplace = True)

In [ ]:
mu = re.mean()
mu
#기대 로그수익률

SSBL    0.000506
HTSL    0.000746
SKH     0.001487
JBS     0.006508
dtype: float64

In [ ]:
muy = re.mean()*230 #1년간 거래일
muy
#년간 기대 로그수익률 추정

SSBL    0.116356
HTSL    0.171513
SKH     0.341979
JBS     1.496893
dtype: float64

In [ ]:
mu = mu.values
mu
#넘피 어레이 형태로 바꾸어줌

array([0.0005059 , 0.00074571, 0.00148687, 0.00650823])

In [ ]:
port_re = pf.dot(mu)
port_re
#pf의 기대수익률 = port_re

array([0.00231168, 0.00375122, 0.00413128])

In [ ]:
re.std()
#일별 수익률의 표준편차를 나타냄
#변동성이 클 수록 위험이 큼
#년간 계산하고 싶으면 루트(230)을 곱함= np.sqrt(230)


SSBL    0.023276
HTSL    0.023873
SKH     0.022284
JBS     0.046287
dtype: float64

In [ ]:
cov_mat = re.cov()
cov_mat = cov_mat.values #레이어로 저장
cov_mat
#단위를 년 환산하기 위해 230곱함
#공분산이 기업별 리스크

array([[5.41781256e-04, 4.56511227e-05, 2.55312796e-05, 1.82188579e-04],
       [4.56511227e-05, 5.69901068e-04, 1.02422060e-04, 1.09117587e-04],
       [2.55312796e-05, 1.02422060e-04, 4.96584379e-04, 2.22129994e-04],
       [1.82188579e-04, 1.09117587e-04, 2.22129994e-04, 2.14245490e-03]])

In [ ]:
port_var = np.dot(np.dot(pf, cov_mat), pf.T) 
port_std = np.sqrt(port_var) 
print("Risk: ", port_std)
#포트폴리오의 리스크

Risk:  [[0.01789693 0.02086991 0.02187915]
 [0.02086991 0.02606147 0.0275795 ]
 [0.02187915 0.0275795  0.03011917]]


In [ ]:
print(mu[0],mu[1],mu[2],mu[3])
print(port_re)
#'SSBL','HTSL','SKH','JBS'의 기대수익률
#포트폴리오 기대수익률

0.0005058951899011879 0.0007457071417695828 0.0014868657256615746 0.006508232600708729
[0.00231168 0.00375122 0.00413128]


In [ ]:
print(np.sqrt(cov_fn[0][0]),np.sqrt(cov_fn[1][1]),
                                   np.sqrt(cov_fn[2][2]), np.sqrt(cov_fn[3][3]))
#'SSBL','HTSL','SKH','JBS'의 리스크

0.023225539499939024 0.023820647290496173 0.022235676668758184 0.046185927330237284


In [ ]:
port_std
#포트폴리오의 리스크 1*1, 2*2, 3*3

array([[0.01789693, 0.02086991, 0.02187915],
       [0.02086991, 0.02606147, 0.0275795 ],
       [0.02187915, 0.0275795 , 0.03011917]])